<a href="https://colab.research.google.com/github/BanJeoker/CSCI6511_AI_Projects/blob/master/React/ReAct_Implementation_v1_Oct08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Youtube template**

*   https://www.youtube.com/watch?v=hKVhRA9kfeM&t=2901s
*   https://github.com/alejandro-ao/agents-no-framework/blob/master/agent-w-loop.ipynb


#### Key findings

* ReAct is not very difficult to implement. We can totally do it without using langchain. The main thing is how powerful the tools are.

* on some simple multi-hop reasoning questions, the ReAct does perform very well right now

* The success of ReAct largely depends on the quality of the RAG, because LLM will deem the returned chunks as ground truth, hence
    * the number of chunks returned matters
    * chunk size matters
    * embeddings or query method decides the relevancy of the returned chunks

* LLM will first think of tools to use before checking what tools are available, if the desired tool is not present/defined, it will use its internal knowledge to solve it.

* sometimes it is hard to trigger the LLM to use defined tools, it might directly use its internal knowledge. Mabye need to work on a more stringent prompt.

* the result might be different each time you run it

* intuition: the system prompt is very important, it is good practice to write a few examples tailored to our use case

#### dependencies

In [ ]:
!pip -q install langchain_openai openai python-dotenv langchain langchain_community pypdf sentence-transformers PyPDF2 pycryptodome tiktoken faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78

In [ ]:
%%writefile .env
OPENAI_API_KEY=""


Overwriting .env


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#### test openai connection

In [3]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


messages = [
        {"role": "system", "content": 'You are a sport fanatic'},
        {"role": "user", "content": 'how many superbowls has Rams won'},
    ]
response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
response_message = response.choices[0].message.content
print(response_message )

As of October 2023, the Los Angeles Rams have won two Super Bowls. Their victories came in Super Bowl XXXIV, when they were known as the St. Louis Rams, and in Super Bowl LVI, after they had returned to Los Angeles.


#### build RAG search tool

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import FAISS


In [4]:
pdf_reader = PdfReader('/content/naf23.pdf')
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [5]:
def process_text(text, chunk_size=1000, chunk_overlap=100):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    embeddings = OpenAIEmbeddings()
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    return knowledgeBase

knowledgeBase = process_text(text)

In [6]:
def search(query):
    return knowledgeBase.similarity_search(query, k=2)

In [7]:
search('total revenue')

[Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                        548,618           \nTotal Supporting Services 1,086,697       -                        1,086,697       \nTotal Expenses 4,996,458       -                        4,996,458'),
 Document(metadata={}

these two chunks are correct, one for 2022, one for 2023, however, the exact year is cut off from the chunk, as the chunk size is only 1k characters

#### building ReAct agent and test it step by step

In [13]:
client

In [14]:
class Agent:
    def __init__(self, client=client, system_prompt: str = "") -> None:
        self.client = client # the llm
        self.system_prompt = system_prompt
        self.messages: list = []
        if self.system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=self.messages,
            temperature=0
        )
        return completion.choices[0].message.content

In [9]:
system_prompt = """
you are a financial expert and an underwriter working for commerical banking of a bank.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search:
e.g. search: total income
Runs a serach function in the knowledge base and return relevant information as chunks of text.


Example session:

Question: analyze financial performance of Apple Inc and potential drivers?
Thought: to analyse finanial performance, I need to know Apple's total revenue, so I need to search its total revenue
Action: search: Apple Inc total revenue
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Total Revenue 979,651...'

Thought: to know the potential drivers, I need to know the buisness composition of Apple, so I need to search its buisness composition
Action: search: Apple Inc buisness composition
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Apple's business composition includes iPhone: Major revenue driver. Mac: Laptops and desktops...'

If you have the answer, output it as the Answer.

Answer: Apple's financial performance is strong with a total revenue of 979,651, which is drived by iPhone sales.

Now it's your turn:
""".strip()

In [ ]:
agent = Agent(client=client, system_prompt=system_prompt)
result = agent("what is total revenue in 2023")
print(result)

Thought: To provide the total revenue for 2023, I need to search for the most recent financial data or reports that include this information.
Action: search: total revenue 2023
PAUSE


In [ ]:
result = search('total revenue 2023')
print(result)

[Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                        548,618           \nTotal Supporting Services 1,086,697       -                        1,086,697       \nTotal Expenses 4,996,458       -                        4,996,458'), Document(metadata={},

In [ ]:
next_prompt = "Observation: {}".format(result)
next_prompt

"Observation: [Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \\nNet Assets Released from Restrictions 975,954           (975,954)          -                        \\nTotal Support and Revenue 3,546,026       638,761           4,184,787       \\nExpenses\\nProgram services\\nResearch 1,574,354       -                        1,574,354       \\nEducation and service 1,146,980       -                        1,146,980       \\nDrug Development Collaborative 1,188,427       -                        1,188,427       \\nTotal Program Services 3,909,761       -                        3,909,761       \\nSupporting services\\nManagement and general 538,079           -                        538,079           \\nFundraising 548,618           -                        548,618           \\nTotal Supporting Services 1,086,697       -                        1,086,697       \\nTotal Expenses 4,996,458       -                        4,996,4

In [ ]:
result = agent(next_prompt)
print(result)


Thought: The search results provide two different figures for total revenue in 2023, which are 979,651 and 981,690. These figures might be from different sources or contexts. To provide a precise answer, I need to clarify the context or source of these figures. However, based on the available data, the total revenue for 2023 is approximately in the range of 979,651 to 981,690.

Answer: The total revenue for 2023 is approximately between 979,651 and 981,690, based on the available data.


there are two chunks returned, the LLM does not know which is correct

In [ ]:
for msg in agent.messages:
    print(msg['content'])

you are a financial expert and an underwriter working for commerical banking of a bank.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search:
e.g. search: total income
Runs a serach function in the knowledge base and return relevant information as chunks of text.


Example session:

Question: analyze financial performance of Apple Inc and potential drivers?
Thought: to analyse finanial performance, I need to know Apple's total revenue, so I need to search its total revenue
Action: search: Apple Inc total revenue
PAUSE 

You will be called again with this:

Observation: Document(metadata={}, page_content='Total Revenue 979,651...' 

Thought: to know the potential drivers, I need to know the buisn

#### automate the ReAct by writing a while loop

In [10]:
tools=["search"]

In [32]:
import re
def react_run(max_iterations=10, query: str = "", tools=tools, system_prompt=system_prompt):

    agent = Agent(client=client, system_prompt=system_prompt)
    tools = tools

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [ ]:
react_run(query="what is National Ataxia Foudnations's revenue 2023?", tools=tools, system_prompt=system_prompt)

Thought: To answer the question about the National Ataxia Foundation's revenue for 2023, I need to search for their financial information for that year.
Action: search: National Ataxia Foundation revenue 2023
PAUSE
Observation: [Document(metadata={}, page_content="Research grants 1,011,330       -                        752,261           1,763,591       1,558                -                        1,765,149       \nSupport grants -                        3,071                -                        3,071                -                        -                        3,071                \n \nTotal Expenses 1,837,520$     579,104 $         943,904 $         3,360,528$      377,042 $         393,619 $         4,131,189$      National Ataxia Foundation\nStatements of Functional Expenses (Continued)\nFor the Years Ended December 31, 2023 and 2022\n2022\nProgram Services Supporting Services\nSee Independent Auditor's Report and Notes to the Financial Statements.\n102023 2022\nCash Flows

**this is the problem of RAG, it returns chunks that have similar embeddings with "National Ataxia Foundation", not with "revenue"**

In [ ]:
react_run(query="what is the difference between total revenue in 2022 and 2023, is it an increase or decrease?", tools=tools, system_prompt=system_prompt)


Thought: To determine the difference in total revenue between 2022 and 2023, I need to find the total revenue for both years and compare them.
Action: search: total revenue 2022
PAUSE
Observation: [Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                      

ground truth:
* total revenue 2022: 979,651
* total revenue 2023: 981,690

the RAG returns two document chunks, both have total revenue, one is 2022, one is 2023. However, the exact year is not in the chunks, so the LLM assumes the order of the documents is the order of the year. Hence, the second query is correct.
*  RAG is important in the quality of the result, should increase the chunk size
* the LLM want to use 'calculate' tool, but we do not define a 'calculate' tool, so it uses its own knowledge to calculate it.

#### same query, but this time we define 'calculate'

In [ ]:
def calculate(operation: str) -> float:
    return eval(operation)

In [18]:
system_prompt = """
you are a financial expert and an underwriter working for commerical banking of a bank.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search:
e.g. search: total income
Runs a serach function in the knowledge base and return relevant information as chunks of text.

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

Example session:

Question: analyze financial performance of Apple Inc and potential drivers?
Thought: to analyse finanial performance, I need to know Apple's total revenue, so I need to search its total revenue
Action: search: Apple Inc total revenue
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Total Revenue 979,651...'

Thought: to know the potential drivers, I need to know the buisness composition of Apple, so I need to search its buisness composition
Action: search: Apple Inc buisness composition
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Apple's business composition includes iPhone: Major revenue driver. Mac: Laptops and desktops...'

If you have the answer, output it as the Answer.

Answer: Apple's financial performance is strong with a total revenue of 979,651, which is drived by iPhone sales.

Now it's your turn:
""".strip()

In [52]:
tools=["search","calculate"]

In [ ]:
react_run(query="what is the difference between total revenue in 2022 and 2023, is it an increase or decrease?", tools=tools, system_prompt=system_prompt)

Thought: To determine the difference in total revenue between 2022 and 2023, I need to find the total revenue for both years and compare them.
Action: search: total revenue 2022
PAUSE
Observation: [Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                      

In [ ]:
eval("981,690 - 979,651")

(981, -289, 651)

**this is the reason why we need to define proper tools instead of relying on eval to calculate stuff**

#### same query, let us increase the size of the  chunk to make sure it contains the year

In [ ]:
knowledgeBase = process_text(text, chunk_size=2000)
def search(query):
    return knowledgeBase.similarity_search(query, k=2)

In [ ]:
react_run(query="what is the difference between total revenue in 2022 and 2023, is it an increase or decrease?", system_prompt=system_prompt)

Thought: To determine the difference in total revenue between 2022 and 2023, I need to find the total revenue for both years and compare them.
Action: search: total revenue 2022
PAUSE
Observation: [Document(metadata={}, page_content="Total Lease Liabilities 58,103 $          \n \n19 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 6: Net Assets with Donor Restrictions  \n \nNet assets with donor restrictions were available for the following purposes at December 31:  \n \n2023 2022\nSCA2 Fund 880,250 $        500,000 $        \nMR Imaging Study Fund 612,044           -                        \nGeneral Research (Includes stock) 394,144           385,387           \nCSO Funding 360,800           450,588           \nSCA3 Clementz 291,120           340,604           \nAAC Travel Grant 122,635           138,542           \nSCA3 13,143             138,333           \nMSA 3,848               3,848               \nFriedreich's Fund 2,2

**The RAG fails agian**

**RAG fails to retrieve the correct chunks**

#### same query, but this time, only RAG only returns 1 document chunk from retrieval, see whether LLM will first search 2022, then 2023, use 1000 chunk size again.

In [ ]:
knowledgeBase = process_text(text, chunk_size=1000)
def search(query):
    return knowledgeBase.similarity_search(query, k=1)

In [49]:
react_run(query="what is the difference between total revenue in 2022 and 2023, is it an increase or decrease?", tools=tools, system_prompt=system_prompt)

Thought: To determine the difference in total revenue between 2022 and 2023, I need to find the total revenue for both years and compare them.
Action: search: total revenue 2022
PAUSE
Observation: [Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                      

* the LLM does first search 2022, then search 2023.
* <font color='red'>when only 1 document chunk is returned, the LLM assume that is the correct answer, when two searches both return the same result, the LLM think there is no difference.</font>

#### same query, this time do not provide relevant example in the system_prompt

In [11]:
tools=["search","calculate"]

In [12]:
knowledgeBase = process_text(text, chunk_size=1000)
def search(query):
    return knowledgeBase.similarity_search(query, k=1)

def calculate(operation: str) -> float:
    return eval(operation)

In [15]:
system_prompt = """
you are a financial expert and an underwriter working for commerical banking of a bank.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search:
e.g. search: total income
Runs a serach function in the knowledge base and return relevant information as chunks of text.


Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [17]:
react_run(query="what is the difference between total revenue in 2022 and 2023, is it an increase or decrease?", tools=tools, system_prompt=system_prompt)

Thought: I need to find the total revenue for both 2022 and 2023 to determine the difference and whether it is an increase or decrease.
Action: search: total revenue 2022
PAUSE
Observation: [Document(metadata={}, page_content='Total Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079           \nFundraising 548,618           -                        548,6

**after changing the prompt to the examples from the youtube video, which is a totally different example using different method. The ReAct still works, but this time it is more intelligent, it found out that the returned chunks are the same during each iteration, so no answer can be given**

#### additional test on multi-hop reasoning

In [21]:
question='What is the minimum cost of an item required for it to be classified as equipment according to the accounting rule'
react_run(query=question, tools=tools, system_prompt=system_prompt)

Thought: To answer this question, I need to find the accounting rule that specifies the minimum cost for an item to be classified as equipment. I will search for this information.
Action: search: minimum cost of an item to be classified as equipment according to accounting rules
PAUSE
Observation: [Document(metadata={}, page_content='Note 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nProperty and equipment are capitalized at cost. Depreciation is provided for using the straight -line method over the \nestimated useful life of the asset. The Foundation considers items with a cost greater than $2,500 and a useful life \ngreater than one year to be property and equipment. Items below $2,500 are expensed when purchased.  \n \nK. Functional Expenses  \n \nThe costs of providing program and other activities have been summarized on a functional basis in the statements of \nactivities. Accordingly, certain costs have been allocated among the p

In [22]:
question='Mary bought a $1k computer for the company, can it be considered as an equipment'
react_run(query=question, tools=tools, system_prompt=system_prompt)

Thought: To determine if the $1,000 computer can be considered as equipment, I need to understand the accounting definition of equipment and the criteria used to classify an asset as equipment. Typically, equipment is a long-term asset used in operations, but I need to confirm this with specific criteria or guidelines.

Action: search: accounting definition of equipment
PAUSE
Observation: [Document(metadata={}, page_content='Note 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nProperty and equipment are capitalized at cost. Depreciation is provided for using the straight -line method over the \nestimated useful life of the asset. The Foundation considers items with a cost greater than $2,500 and a useful life \ngreater than one year to be property and equipment. Items below $2,500 are expensed when purchased.  \n \nK. Functional Expenses  \n \nThe costs of providing program and other activities have been summarized on a functional basis 

**the ReAct agent successfully demonstrates the ability to do simple multihop reasoning**

In [23]:
question='Mary bought a $3k computer for the company, where should this money go on the income statement'
react_run(query=question, tools=tools, system_prompt=system_prompt)

Thought: To determine where the purchase of a computer should be recorded on the income statement, I need to understand the nature of the transaction and how it affects the financial statements. The purchase of a computer is a capital expenditure and typically does not directly appear on the income statement. Instead, it is capitalized as an asset and then depreciated over time. However, the depreciation expense related to the computer will appear on the income statement. I will search for information on how capital expenditures and depreciation are treated in financial statements.

Action: search: capital expenditure and depreciation treatment in financial statements
PAUSE
Observation: [Document(metadata={}, page_content='Note 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nProperty and equipment are capitalized at cost. Depreciation is provided for using the straight -line method over the \nestimated useful life of the asset. The Found

#### try implement a function called check_value, to trigger LLM to check the price of certain items

In [29]:
def check_value(name: str) -> float:
    if name=='Mac':
        return 3000
    elif name=='iphone':
        return 1000
    else:
        return 500
tools=["search","calculate","check_value"]

system_prompt = """
you are a financial expert and an underwriter working for commerical banking of a bank.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search:
e.g. search: total income
Runs a serach function in the knowledge base and return relevant information as chunks of text.

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

check_value:
e.g., check_value: iMac
Runs a check_value function that return the price/value of an item in dollars

Example session:

Question: analyze financial performance of Apple Inc and potential drivers?
Thought: to analyse finanial performance, I need to know Apple's total revenue, so I need to search its total revenue
Action: search: Apple Inc total revenue
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Total Revenue 979,651...'

Thought: to know the potential drivers, I need to know the buisness composition of Apple, so I need to search its buisness composition
Action: search: Apple Inc buisness composition
PAUSE

You will be called again with this:

Observation: Document(metadata={}, page_content='Apple's business composition includes iPhone: Major revenue driver. Mac: Laptops and desktops...'

If you have the answer, output it as the Answer.

Answer: Apple's financial performance is strong with a total revenue of 979,651, which is drived by iPhone sales.

Now it's your turn:
""".strip()


In [30]:
question='Marry bought a Mac for the company, where should this money go on the income statement'
react_run(query=question, tools=tools, system_prompt=system_prompt)

Thought: To determine where the purchase of a Mac should be recorded on the income statement, I need to understand the nature of the transaction and how it affects the company's financials. The purchase of a Mac is a capital expenditure, which typically does not appear directly on the income statement. Instead, it is recorded as an asset on the balance sheet and then depreciated over time. The depreciation expense is what appears on the income statement. I will confirm this by searching for information on how capital expenditures and depreciation are recorded in financial statements.

Action: search: capital expenditure and depreciation on income statement
PAUSE
Observation: [Document(metadata={}, page_content='Note 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nProperty and equipment are capitalized at cost. Depreciation is provided for using the straight -line method over the \nestimated useful life of the asset. The Foundation consid

**the LLM does not use our method, it just uses it internal knowledge to determine the price of Mac**

In [33]:
question='Marry bought a Mac of unknown value for the company, where should this money go on the income statement'
react_run(query=question, tools=tools, system_prompt=system_prompt)

Thought: To determine where the money spent on purchasing a Mac should go on the income statement, I need to understand the nature of the purchase and how it is typically accounted for in financial statements. Generally, purchases of equipment like a Mac are considered capital expenditures and are not directly recorded on the income statement. Instead, they are capitalized and depreciated over time. However, the depreciation expense related to the Mac would appear on the income statement. I will confirm this by checking the treatment of such purchases in financial accounting.

Action: search: treatment of equipment purchase in financial accounting
PAUSE
Observation: [Document(metadata={}, page_content='Note 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nProperty and equipment are capitalized at cost. Depreciation is provided for using the straight -line method over the \nestimated useful life of the asset. The Foundation considers items